In [ ]:
path = []


In [2]:
path = [[0, 10], [1, 9], [2, 9], [3, 10], [4, 9], [5, 10], [6, 9], [7, 9], [8, 10], [9, 10], [10, 11], [11, 10], [12, 10], [13, 11],[14, 11], [15, 11], [16, 10], [17, 10], [18, 9], [19, 10]]

In [19]:
import cv2
import numpy as np
blank = cv2.imread("blank.png",0)
blank = cv2.resize(blank,(20,20))
# path = np.array(path)
# epsilon = 0.1*cv2.arcLength(path, True)
# path = cv2.approxPolyDP(path, epsilon, False)
# print(path)
pnts = []
for i in range(1,16):
    pts = [np.mean(path[0:15*(i)][0]),np.mean(path[0:15*(i)][1])]
    pnts.append(pts)
#     cv2.line(blank,(path[i][1],path[i][0]),(path[i+1][1],path[i+1][0]),0)
#     pnts = np.mean(path[i:i+15][0])
print(pnts)
blank = cv2.resize(blank,(680,480))
cv2.imshow("blank",blank)
q = cv2.waitKey(1)
if q == 27:
    cv2.destroyAllWindows()

[[5.0, 5.0], [7.5, 7.5], [10.5, 10.5]]


In [32]:
import math
slp = []
for pix in pnts:
    slp.append(math.atan2(340-pix[1],pix[0]-240) - 1.57)

In [33]:
import math
print(slp)

[1.1292184306130484, 0.0007963267948964958]


In [84]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb

In [ ]:
#output:: 6 category  
#input:: 15 slope 
#mode = xgboost


In [111]:
import random
from collections import deque
class DQNAgent:
    def __init__(self):
        self.state_size = 15
        self.action_size = 6
        self.memory = deque(maxlen=pow(10,6))
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.bst = xgb.Booster({'nthread':4})
        self.name = 'steerDeepQL'
        self.modelname = 'xgbmodel'
    
    def xgbmodel_bld(self,data,label):
        param = {'max_depth': 3, 'eta': 0.3, 'silent': 1, 'objective': 'multi:softprob','num_class': 6}  
        param['nthread'] = 4
        num_round = 2000  
        dtrain = xgb.DMatrix(data, label=label)
        self.bst = xgb.train(param, dtrain, num_round, xgb_model = self.name)
        self.save_bst()
        
    def load_bst(self):
        self.bst.load_model(self.name)
    
    def save_bst(self):
        self.bst.save_model(self.name)
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.bst.predict(xgb.DMatrix(state))
        return np.argmax(act_values[0])  
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        minibatch = self.memory
        print(len(minibatch))
        self.memory.clear()
        for state, action, reward, next_state, done in minibatch:
            target = self.bst.predict(xgb.DMatrix(state))
            
            if done:
                target[0][action] = reward
            else:
                t = self.bst.predict(xgb.DMatrix(next_state))[0]
                target[0][action] = reward + self.gamma * np.amax(t)
            print(target)
#         self.xgbmodel_bld(state, target)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def decl_rew(self,state):
        rew = np.sum((np.array([1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75]) - abs(state))*np.array([10,10,10,9,9,9,8,8,7,6,5,4,3,2,1]))
        done = False
        if rew > 130:
            done = True
        return rew,done
    
    def get_nextstate(self):
        ########
        stt = np.zeros((15,), dtype=float)
        return stt
    
def steer(GameOn):
    state_size = 15
    action_size = 6
    agent = DQNAgent()
    batch_size = 50
    state = np.zeros((15,), dtype=float)
    state = agent.get_nextstate()
    agent.load_bst()
    
    while GameOn:
        state = np.reshape(state, [1, state_size])
        action = agent.act(state)
#         print("action:",action)
#         execute_action(action)
        reward, done = agent.decl_rew(state)
        reward = reward if not done else -10
        next_state = agent.get_nextstate()
        if(next_state == []):
            continue;
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
#         if done:
#             print("updating weights under 10 deg")
#             agent.update_target_model()
        if len(agent.memory) >= 500:
            print("replaying at {} xgboost".format(len(agent.memory)))
            agent.replay()
    
        

In [112]:
steer(True)

/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:86: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 500 xgboost
500
replaying at 5

KeyboardInterrupt: 

In [95]:
data = np.random.rand(1,15) # 5 entities, each contains 10 features
label = np.random.randint(6, size=0)
dtest = xgb.DMatrix(data)
print(dtest)
# DQNAgent().xgbmodel_bld(data,label)
DQNAgent().bst.predict(dtest)

array([ 0.5], dtype=float32)

In [83]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [89]:
import random
import gym
import numpy as np
from collections import deque
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.optimizers import Adam
from keras import backend as K

EPISODES = 500


class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=pow(10,6))
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()
        
    
    def _build_model(self):
        param = { 'max_depth': 3, 'eta': 0.3, 'silent': 1, 'objective': 'multi:softprob','num_class': 6}  
        num_round = 2000  
        classifier = xgb.sklearn.XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
                       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
                        min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
                       objective='multi:softmax', reg_alpha=0, reg_lambda=1,
                       scale_pos_weight=1, seed=42, silent=True, subsample=1)
        return classifier
    
    def update_target_model(self):
        self.target_model.load_model(self.model.save_model())
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0]) 
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
#         eval_set = [(X_test, y_test)]
        self.model.fit(state, target,eval_metric="error", eval_set=eval_set, verbose=True)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_model(name)
    
    def save(self, name):
        self.model.save_model(name)
    
    def decl_rew(self,state):
        rew = np.sum((np.array([1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75]) - abs(state))*np.array([10,10,10,9,9,9,8,8,7,6,5,4,3,2,1]))
        d = False
        if rew > 130:
            d = True
        return [nxtstate, rew, True]
        


def steer():
    state_size = 15
    action_size = 6
    agent = DQNAgent(state_size, action_size)
    
#     agent.load_model("./save/steerwts.h5")
    
    batch_size = 50
    state = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] 
    while True:
        done = False
        while True:
            state = np.reshape(state, [1, state_size])
            action = agent.act(state)
            print("action:",action)
            execute_action(action)
            next_state, reward, done = decl_rew(state)##input and reward
            if(next_state == []):
                continue;
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print("updating weights under 10 deg")
                agent.update_target_model()
            if len(agent.memory) > batch_size:
                print("replaying at 50 and saving weights")
                agent.replay(batch_size)

In [12]:
import numpy as np
from PIL import ImageGrab
import cv2
import time

def screen_record(): 
    last_time = time.time()
    while True:
        # 800x600 windowed mode
        printscreen_pil =  ImageGrab.grab(bbox=(0,40,800,640))
        printscreen_numpy =   np.array(printscreen_pil.getdata(),dtype='uint8')\
        .reshape((printscreen_pil.size[1],printscreen_pil.size[0],3))
        print('loop took {} seconds'.format(time.time()-last_time))
        last_time = time.time()

    ##    cv2.imshow('window',cv2.cvtColor(printscreen_numpy, cv2.COLOR_BGR2RGB))
    ##    if cv2.waitKey(25) & 0xFF == ord('q'):
    ##        cv2.destroyAllWindows()
    ##        break

In [12]:
import numpy as np
import xgboost as xgb
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn.datasets import dump_svmlight_file
from sklearn.externals import joblib
from sklearn.metrics import precision_score

iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

param = {
    'max_depth': 3, 'eta': 0.3, 'silent': 1, 'objective': 'multi:softprob','num_class': 3}  
num_round = 200  
bst = xgb.train(param, dtrain, num_round,evals=[(dtrain, "training")],xgb_model=self.name )
# preds = bst.predict(dtest)

# best_preds = np.asarray([np.argmax(line) for line in preds])
# print("Numpy array precision:", precision_score(y_test, best_preds, average='macro') )


# joblib.dump(bst, 'bst_modellll', compress=True)



XGBoostError: b'[02:39:16] src/io/local_filesys.cc:154: Check failed: allow_null  LocalFileSystem: fail to open "mm"\n\nStack trace returned 6 entries:\n[bt] (0) 0   libxgboost.dylib                    0x0000000118a6faa8 _ZN4dmlc15LogMessageFatalD2Ev + 40\n[bt] (1) 1   libxgboost.dylib                    0x0000000118b3bb02 _ZN4dmlc2io15LocalFileSystem4OpenERKNS0_3URIEPKcb + 962\n[bt] (2) 2   libxgboost.dylib                    0x0000000118b35911 _ZN4dmlc6Stream6CreateEPKcS2_b + 65\n[bt] (3) 3   libxgboost.dylib                    0x0000000118a86131 XGBoosterLoadModel + 33\n[bt] (4) 4   _ctypes.cpython-36m-darwin.so       0x00000001064122b7 ffi_call_unix64 + 79\n[bt] (5) 5   ???                                 0x00007fff5aeb9cb0 0x0 + 140734718778544\n'

In [146]:
xgb1 = joblib.load('bst_model')
xgb.train(param, dtrain, num_round)#,xgb_model='bst_model')
xgb1.predict(dtest)
# joblib.dump(xgb1, 'bst_modell.pkl', compress=True)

XGBoostError: b'[02:20:01] src/objective/multiclass_obj.cc:75: Check failed: label_error >= 0 && label_error < nclass SoftmaxMultiClassObj: label must be in [0, num_class), num_class=2 but found 2 in label.\n\nStack trace returned 6 entries:\n[bt] (0) 0   libxgboost.dylib                    0x000000011950eaa8 _ZN4dmlc15LogMessageFatalD2Ev + 40\n[bt] (1) 1   libxgboost.dylib                    0x00000001195823b2 _ZN7xgboost3obj20SoftmaxMultiClassObj11GetGradientERKNSt3__16vectorIfNS2_9allocatorIfEEEERKNS_8MetaInfoEiPNS3_INS_6detail18bst_gpair_internalIfEENS4_ISE_EEEE + 866\n[bt] (2) 2   libxgboost.dylib                    0x000000011950b0f6 _ZN7xgboost11LearnerImpl13UpdateOneIterEiPNS_7DMatrixE + 1014\n[bt] (3) 3   libxgboost.dylib                    0x000000011952473f XGBoosterUpdateOneIter + 79\n[bt] (4) 4   _ctypes.cpython-36m-darwin.so       0x00000001063a22b7 ffi_call_unix64 + 79\n[bt] (5) 5   ???                                 0x00007fff5af2a120 0x0 + 140734719238432\n'

In [85]:
bst.save_model('m')

In [86]:
bst1 = xgb.Booster({'nthread':4}) #init model
bst1.load_model("m")

In [89]:
bst1.predict(dtest)

array([[  1.95936323e-03,   9.79995668e-01,   1.80449765e-02],
       [  9.90137100e-01,   9.55410488e-03,   3.08829825e-04],
       [  5.22883493e-05,   6.85594045e-04,   9.99262154e-01],
       [  3.54431290e-03,   9.92628455e-01,   3.82724102e-03],
       [  2.24673119e-03,   9.37656224e-01,   6.00970164e-02],
       [  9.96158957e-01,   3.53033794e-03,   3.10708099e-04],
       [  5.29642683e-03,   9.93998289e-01,   7.05274229e-04],
       [  1.53118500e-03,   1.45450290e-02,   9.83923852e-01],
       [  3.95626499e-04,   9.85751688e-01,   1.38526373e-02],
       [  8.96925747e-04,   9.98504043e-01,   5.99057821e-04],
       [  1.75884133e-03,   1.67075731e-02,   9.81533587e-01],
       [  9.97719586e-01,   1.89933134e-03,   3.81007325e-04],
       [  9.90506113e-01,   9.18492302e-03,   3.08944931e-04],
       [  9.97789264e-01,   1.89946394e-03,   3.11216601e-04],
       [  9.97789264e-01,   1.89946394e-03,   3.11216601e-04],
       [  4.74228611e-04,   9.99225855e-01,   2.9993546

In [ ]:
bst1